In [1]:
import numpy as np
from collections import deque, defaultdict
import heapq
from copy import deepcopy

In [2]:
test = ['jqt: rhn xhk nvd',
        'rsh: frs pzl lsr',
        'xhk: hfx',
        'cmg: qnr nvd lhk bvb',
        'rhn: xhk bvb hfx',
        'bvb: xhk hfx',
        'pzl: lsr hfx nvd',
        'qnr: nvd',
        'ntq: jqt hfx bvb xhk',
        'nvd: lhk',
        'lsr: lhk',
        'rzs: qnr cmg lsr rsh',
        'frs: qnr lhk lsr']

data = np.genfromtxt('day25_input.txt', dtype=str, delimiter='\n', comments=None)

In [3]:
def gen_graph(data):
    graph = {}
    for line in data:
        left, right = line.split(': ')
        right = right.split()
        graph[left] = right
        
    #get missing keys
    to_add = []
    for key, value in graph.items():
        for v in value:
            if v not in graph.keys():
                to_add.append(v)
    for v in to_add:
        graph[v] = []
        
    #get missing connections
    for key1 in graph.keys():
        for key2 in graph.keys():
            if key1 == key2:
                continue
                
            if key1 in graph[key2]:
                graph[key1].append(key2)
        
    return graph

def BFS(graph, start, end):
    queue = deque([start])
    dist = {start: [0, [start]]}
    
    while len(queue):
        cur_pos = queue.popleft()
        cur_dist, cur_path = dist[cur_pos]
        nxt_dist = cur_dist + 1
        
        for nxt_pos in graph[cur_pos]:
            if nxt_pos not in dist.keys():
                queue.append(nxt_pos)
                
                nxt_path = deepcopy(cur_path)
                nxt_path.append(nxt_pos)
                dist[nxt_pos] = [nxt_dist, nxt_path]
                
    return dist[end][1]

def dijkstra(graph, start, end):
    dist = {}
    dist[start] = 0
    
    routes = {}
    routes[start] = [start]
    
    queue = [(0,start)]
    while queue:
        cur_dist, cur_pos = heapq.heappop(queue)
        nxt_dist = cur_dist+1
        
        for nxt_pos in graph[cur_pos]:
            if nxt_pos not in dist.keys():
                dist[nxt_pos] = cur_dist
                heapq.heappush(queue, (nxt_dist, nxt_pos))
                rout = deepcopy(routes[cur_pos])
                rout.append(nxt_pos)
                routes[nxt_pos] = rout
            
    return routes[end]

def BFS_cut(graph, start, cut_wires):
    queue = deque([start])
    dist = {start: 0}
    
    while len(queue):
        cur_pos = queue.popleft()
        cur_dist = dist[cur_pos]
        nxt_dist = cur_dist+1
        for nxt_pos in graph[cur_pos]:
            if nxt_pos not in dist.keys() and\
               (nxt_pos, cur_pos) not in cut_wires and (cur_pos, nxt_pos) not in cut_wires:
                queue.append(nxt_pos)
                dist[nxt_pos] = nxt_dist
                
    return list(dist.keys())
        
def split_graph(data):
    graph = gen_graph(data)
    
    nodes = list(graph.keys())
    
    wires = defaultdict(int)
    
    np.random.seed(256)
    for i in range(0, 100):
        start, end = np.random.choice(nodes, 2, False)
        path = BFS(graph, start, end)
        #path = dijkstra(graph, start, end)
        
        for j in range(0, len(path)-1):
            wires[(path[j],path[j+1])] += 1
            
    keys = []
    values = []
    for key, value in wires.items():
        keys.append(key)
        values.append(value)
        
    idx = np.argsort(values)[::-1][:3]
    cut_wires = [keys[idx[0]], keys[idx[1]], keys[idx[2]]]
    
    side0 = BFS_cut(graph, nodes[0], cut_wires)
    side1 = np.setdiff1d(nodes, side0)
    
    return len(side0)*len(side1)
    
print(split_graph(test))
print('Part 1 result:', split_graph(data))

54
Part 1 result: 582626
